In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class NN:
    def __init__(self):
        self.layersOfWeights = [] #Слои нейронов
        self.layersOfBiases = [] #Слои смещений
        self.activFunc = [] #Слои функций активации (Мало ли они будут разные)
        self.gradActivFunc = [] #Производная от слоя активации. Нужна на случай, если таковой нет(Тут будет просто функция, которая возвращает вход)
        self.countOfLayers = 0 #Просто число слоёв

        self.calculatedBeforeActivFunc = [] #Промежуточные значения до функции активации на каждом слое
        self.calculatedAfterActivFunc = [] #Промежуточные знчаения после вычисления фунцкции активации на каждом слое

        self.errorsDuringTraining = [] #Список ошибок на каждом шагу обучения

    def addLayer(self, n, m):
        """
        n - входное число нейронов<br>
        m - выходное число нейронов
        """
        q = 1
        self.layersOfWeights.append(np.random.uniform(-q, q, (n,m)))
        self.layersOfBiases.append(np.random.uniform(-q, q, (1,m)))
        self.countOfLayers += 1

    def addActivFunc(self, n:int=0):
        """
        n - номер функции активации: <br>
        <b>0</b> - никакой функции активации. Выбор по умолчанию<br>
        <b>1</b> - сигмойда (1/1 + exp(-x))<br>
        <b>2</b> - гиперболический тангенс (tanh(x))<br>
        <b>3</b> - softmax (exp(xi)) / sum(exp(xi))
        """
        none = lambda x: x
        gradNone = lambda x: np.ones_like(x)
        sigmoid = lambda x: 1 / (1 + np.exp(-x))
        gradSigmoid = lambda x: sigmoid(x) * (1 - sigmoid(x))
        tanh = lambda x: np.tanh(x)
        gradTanh = lambda x: 1 - (np.tanh(x))**2

        def softmax(x:np.ndarray):
            """x - вектор предсказаний"""
            x = np.exp(x)
            denuminator = x.sum(axis=1).reshape(x.shape[0], 1)
            return x / denuminator

        def gradSoftmax(b:np.ndarray):
            """Градиент из softmax'a. <br>
            На вход принимает матрицу значений softmax, где строки - это один набор данных, а столбцы - значения выходных нейронов"""
            z = -b[:, :, None] * b[:, None]
            diag = np.diag_indices_from(z[0])
            z[:, diag[0], diag[1]] = b[:] * (1 - b[:])
            return z.sum(axis=1)

        if n ==0:
            func = none
            gradFunc = gradNone
        elif n ==1:
            func = sigmoid
            gradFunc = gradSigmoid
        elif n == 2: 
            func = tanh
            gradFunc = gradTanh
        elif n == 3:
            func = softmax
            gradFunc = gradSoftmax
        else:
            raise Exception("Выбрана неверная функция активации")
        
        self.activFunc.append(func)
        self.gradActivFunc.append(gradFunc)

    def predict(self, data:np.ndarray)->np.ndarray:
        """
        data - данные, на основе которых мы хотим получить предсказание
        """
        self.calculatedAfterActivFunc.append(data)
        a = data @ self.layersOfWeights[0] + self.layersOfBiases[0] #Первый слой
        self.calculatedBeforeActivFunc.append(a)
        a = self.activFunc[0](a)
        self.calculatedAfterActivFunc.append(a)
        
        for i in range(1, self.countOfLayers): #Все последующие слои, начиная со второго
            a = a @ self.layersOfWeights[i] + self.layersOfBiases[i]
            self.calculatedBeforeActivFunc.append(a)
            a = self.activFunc[i](a)
            self.calculatedAfterActivFunc.append(a)

        return a
    
    def _predict(self, data:np.ndarray)->np.ndarray:
        """
        <h3>Этот метод нужен для подсчёта внутри процесса обучения, что бы не влиять на него</h3>
        data - данные, на основе которых мы хотим получить предсказание
        """
        a = data @ self.layersOfWeights[0] + self.layersOfBiases[0] #Первый слой
        a = self.activFunc[0](a)
        if self.countOfLayers != 1:
            for i in range(1, self.countOfLayers): #Все последующие слои, начиная со второго
                a = a @ self.layersOfWeights[i] + self.layersOfBiases[i]
                a = self.activFunc[i](a)
        return a
    
    def _backProp(self, data:np.ndarray, true:np.ndarray, lr):
        """
        Метод обучения. Принимает в себя <b>(потенциально один)</b> экземпляр, на котором и учится 
        """
        self.calculatedBeforeActivFunc.clear()
        self.calculatedAfterActivFunc.clear()
        pred = self._predict(data)

        # e = ((pred-true)**2).mean()

        dedy = (2 * (pred - true).mean()).reshape(1,1) #Это теперь одно число
        self.dedy.append(dedy)

        #Первый слой
        dydz = dedy * self.gradActivFunc[-1](self.calculatedBeforeActivFunc[-1])
        self.dydz.append(dydz)

        dzdw = self.calculatedAfterActivFunc[-2].T @ dydz 
        self.dzdw.append(dzdw)

        dzdb = (dydz * 1).mean(axis=0)
        self.dzdb.append(dzdb)

        dzdy = dydz @ self.layersOfWeights[-1].T #updated
        self.dzdy.append(dzdy)

        self.layersOfWeights[-1] = self.layersOfWeights[-1] - dzdw * lr
        self.layersOfBiases[-1] = self.layersOfBiases[-1] - dzdb * lr

        #Все последующие слои, начиная со второго
        for i in range(1, self.countOfLayers):
            dydz = dzdy * self.gradActivFunc[-i-1](self.calculatedBeforeActivFunc[-i-1])
            dzdw = self.calculatedAfterActivFunc[-i-2].T @ dydz
            dzdb = (dydz * 1).mean(axis=0)
            self.layersOfBiases[-i-1] = self.layersOfBiases[-i-1] - dzdb * lr
            dzdy = dydz @ self.layersOfWeights[-i-1].T
            self.layersOfWeights[-i-1] = self.layersOfWeights[-i-1] - dzdw * lr #updated

            # pred = self._predict(data)
            # e = ((pred-true)**2).mean()
            # self.errorsDuringTraining.append(e)
        
        pred = self._predict(data)
        e = ((pred-true)**2).mean()
        self.errorsDuringTraining.append(e)

    def training(self, data:np.ndarray, true:np.ndarray, steps, lr = 0.001, stopCreteria = 0.001, chart = False):
        self.errorsDuringTraining.clear()

        self.dedy = []
        self.dydz = []
        self.dzdw = []
        self.dzdb = []
        self.dzdy = []

        for i in range(steps):
            self._backProp(data, true, lr)
            if len(self.errorsDuringTraining) >= 2:
                if (self.errorsDuringTraining[-2] - self.errorsDuringTraining[-1]) < stopCreteria:
                    print(f'Всего за {i} шагов модель обучилась достаточно')
                    break
        if chart:
            plt.figure()
            plt.title("График обучения")
            plt.plot(range(self.errorsDuringTraining.__len__()), self.errorsDuringTraining)
            plt.xlabel("Номер итерации")
            plt.ylabel("Ошибка")